In [1]:
import numpy as np

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten


In [3]:
# preparing independent and dependent features
def prepare_data(timeseries,n_features):
    X,y = [], []
    for i in range(len(timeseries)):
        # find the end of this pattern
        end_ix = i + n_features
        # check if are beyond the sequence
        if end_ix > len(timeseries)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = timeseries[i:end_ix], timeseries[end_ix]
        X.append(seq_x)
        y.append(seq_y)

    return np.array(X), np.array(y)

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("../data2.csv")

In [6]:
df.head()

,Unnamed: 0,Name,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,...,Age,Gender,TransactionCount,Duration,Peak Season,Before Season months,Months left for End of Season,AVG Sales,avg order cost,Loan Amount
0,0,Chad Smith,179018.86,177120.88,174173.55,122734.36,362590.35,365079.62,249214.28,242866.21,...,74,1,3273,8,0,3,7,271315,82.894898,570652.374624
1,1,Ronald Collins,42555.73,63249.06,40379.44,57876.43,49185.06,54559.53,51300.02,108893.08,...,58,1,3103,21,1,0,1,60072,19.359330,212107.255479
2,2,Darryl Brown,135601.47,174790.03,133235.49,209354.95,180334.22,144941.36,158670.00,480330.37,...,23,1,4770,5,1,0,0,194417,40.758281,756066.371479
3,3,Becky Williams,145533.08,194858.86,517763.56,587363.81,194260.81,238350.12,138048.70,176593.62,...,34,1,405,6,0,3,1,242533,598.846914,444750.202464
4,4,Jennifer Bruce,111695.66,92337.60,104144.36,89659.33,145544.96,235337.56,276935.12,108437.74,...,48,0,224,11,0,3,1,136020,607.232143,388170.371595


In [7]:
df = df.drop(['Name','Peak Season','Before Season months', 'Months left for End of Season','Gender'],axis = 1)

In [8]:
df = df.drop(['TransactionCount','Unnamed: 0'],axis = 1)

In [9]:
df.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec,Age,Duration,AVG Sales,avg order cost,Loan Amount
0,179018.86,177120.88,174173.55,122734.36,362590.35,365079.62,249214.28,242866.21,436012.52,262280.29,390223.58,294466.47,74,8,271315,82.894898,570652.374624
1,42555.73,63249.06,40379.44,57876.43,49185.06,54559.53,51300.02,108893.08,111362.50,43984.41,60624.84,36887.95,58,21,60072,19.359330,212107.255479
2,135601.47,174790.03,133235.49,209354.95,180334.22,144941.36,158670.00,480330.37,143541.85,131941.96,227762.95,212498.75,23,5,194417,40.758281,756066.371479
3,145533.08,194858.86,517763.56,587363.81,194260.81,238350.12,138048.70,176593.62,230162.27,137788.52,189368.18,160306.05,34,6,242533,598.846914,444750.202464
4,111695.66,92337.60,104144.36,89659.33,145544.96,235337.56,276935.12,108437.74,102481.52,127726.98,120973.45,116962.33,48,11,136020,607.232143,388170.371595


In [10]:
def model_function(x,y):
    model = Sequential()
    model.add(LSTM(50,activation='relu',return_sequences=True, input_shape = (6,1)))
    model.add(LSTM(50,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer = 'adam',loss = 'mse')
    model.fir(x,y,epochs = 50,verbose = 0)

In [11]:
cnt = 0
for index,row in df.iterrows():
    cnt += 1
    if(cnt == 3):
        break
    x = row.iloc[0:12]
    x = x.tolist()
    n_features = 6
    a,b = prepare_data(x,n_features)
    a = a.reshape(a.shape[0],a.shape[1],1)

